In [1]:
import pandas as pd
import os

In [2]:
df=pd.read_excel('spamhamV2.xlsx', sep=',')

In [3]:
df['Email']=df.Email.str.replace(':','')
df['Email']=df.Email.str.replace('"','')
df['Email']=df.Email.str.replace("'","")
df['Email']=df.Email.str.replace('(','')
df['Email']=df.Email.str.replace(')','')
df['Email']=df.Email.str.replace('\n','')
df['Email']=df.Email.str.replace(',','')
df['Email']=df.Email.str.replace('.','')
df['Email']=df.Email.str.lower()


In [4]:
df['Email']=df['Email'].str.split(pat=None,n=-1,expand=False)


In [5]:
dfTreinamento=df.sample(n=None, frac=0.75, replace=False, weights=None, random_state=None, axis=None)
dfTeste=df.sample(n=None, frac=0.25, replace=False, weights=None, random_state=None, axis=None)

In [6]:
dfTreinamento.to_csv('var_Treinamento.csv')
dfTeste.to_csv('var_Teste.csv')

In [7]:
#Calculando a probabilidade de ser Spam dentro do dataframe de Treinamento
Spam=dfTreinamento[dfTreinamento.Class=='spam']
Pspam=len(Spam)/len(dfTreinamento['Class'])
Pspam

0.1323283082077052

In [8]:
#Calculando a probabilidade de ser Ham dentro do dataframe de Treinamento
Ham=dfTreinamento[dfTreinamento.Class=='ham']
Pham=1-Pspam
Pham

0.8676716917922949

In [9]:
dfTotal=dfTreinamento['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
serie = pd.Series(dfTotal)
palavras = serie.value_counts()
#j=len(dfTotal)


In [10]:
# dfTotal=dfTreinamento.sum(axis=None, skipna=None, level=None, numeric_only=None)
# serie = pd.Series(dfTotal)
# palavras = serie.value_counts()

# # dfTotalClass=pd.Series(df['Class'])
# # dfClassSeries=serie.value_counts()
# # #j=len(dfTotal)
# # dfClassSeries

In [11]:
#Calculando a probabilidade de uma palavra acontecer se a mensagem na base treinamento é considerada SPAM
dfTreiSpam=dfTreinamento[dfTreinamento.Class=='spam']
dfTreiSpam=dfTreiSpam['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
SpamSeries=pd.Series(dfTreiSpam)

PS=SpamSeries.value_counts()

#j=len(Spam)+len(Ham)
PalavrasSpam=(PS+1)/(len(SpamSeries)+len(palavras))
#PalavrasSpam

In [12]:
#Calculando a probabilidade de uma palavra acontecer se a mensagem na base treinamento é considerada Ham
dfTreiHam=dfTreinamento[dfTreinamento.Class=='ham']
dfTreiHam=dfTreiHam['Email'].sum(axis=None, skipna=None, level=None, numeric_only=None)
HamSeries=pd.Series(dfTreiHam)

PH=HamSeries.value_counts()

PalavrasHam=(PH+1)/(len(HamSeries)+len(palavras))
#PalavrasHam

In [13]:
# def Bayes (email):
#     var_s=1
#     var_h=1
#     for i in email:
#         if (i in PalavrasSpam) and (i in PalavrasHam):
#             var_s=var_s*((PalavrasSpam[i]+1)/((palavras[i]/j)+j))
#             var_h=var_h*((PalavrasHam[i]+1)/((palavras[i]/j)+j))
#         elif i in PalavrasSpam:
#             var_s=var_s*((PalavrasSpam[i]+1)/((palavras[i]/j)+j))
#             var_h=var_h*(1/j)
#         elif i in PalavrasHam:
#             var_s=var_s*(1/j)
#             var_h=var_h*((PalavrasHam[i]+1)/((palavras[i]/j)+j))
#         else:
#             var_s=var_s*(1/j)
#             var_h=var_h*(1/j)
#     if var_s>=var_h:
#         return 'spam'
#     else:
#         return 'ham'


In [14]:
def Bayes(email):
    
    var_s=1
    var_h=1
    
    for i in email:
        if i in (PalavrasHam):
            var_h*=PalavrasHam[i]
        else:
            var_h*=1/(len(palavras))
    var_h=var_h*(len(dfTreiHam)/len(df))
    
    for i in email:
        if i in PalavrasSpam:
            var_s*=PalavrasSpam[i]
        else:
            var_s*=1/(len(palavras))
    var_s=var_s*(len(df)-len(dfTreiHam))/len(df)
    
    if (var_h>=var_s):
        return 'ham'
    else:
        return 'spam'

In [16]:
for i in dfTeste['Email']:
    dfTeste['Resultados']=Bayes(i)
dfTeste['Resultados'].describe()

count     1393
unique       1
top        ham
freq      1393
Name: Resultados, dtype: object